# Uninflected Sign Systems: Vocabulary-Based Identification

This notebook demonstrates **VocabularyIdentifierScheme** for fixed sign systems.

## Inflected vs Uninflected Sign Systems

| Inflected (English, Russian) | Uninflected (Chinese, Music, Emoji) |
|------------------------------|-------------------------------------|
| Open vocabulary (infinite forms) | Fixed vocabulary (finite signs) |
| Words change form (`run→running`) | Signs are atomic, immutable |
| Typos create different hashes | Each sign is unambiguous |
| Hash-based identification | Direct vocabulary lookup |

**Examples of uninflected sign systems:**
- Chinese/Japanese/Korean characters
- Musical notation (♩♪♫♬)
- Mathematical symbols
- Emoji sequences
- Traffic signs

## Key Insight: HRT is Universal

The **structure** (AM, W, HRT) doesn't change - only the **identification** scheme:

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    HashIdentifierScheme (inflected)                     │
│               content → SHA1 → (reg, zeros) → index                     │
├─────────────────────────────────────────────────────────────────────────┤
│                 VocabularyIdentifierScheme (uninflected)                │
│               sign → vocabulary lookup → index                          │
├─────────────────────────────────────────────────────────────────────────┤
│                          AM / W / HRT                                   │
│               (don't care HOW indices are computed)                     │
└─────────────────────────────────────────────────────────────────────────┘
```

## 1. Setup & Imports

In [1]:
import sys
from pathlib import Path

# Ensure core is importable
PROJECT_ROOT = Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")

Project root: /home/alexmy/SGS/SGS_lib/fractal_manifold/fractal_manifold


In [2]:
# Import core modules
from core.mf_algebra import (
    HashIdentifierScheme,
    VocabularyIdentifierScheme,
    LookupTable,
    START, END,
    build_w_from_am,
    LayerHLLSets,
    cascading_disambiguate,
)
from core.sparse_hrt_3d import Sparse3DConfig, SparseAM3D, Edge3D

# Reload sign_tokenizer to pick up changes
import importlib
import core.sign_tokenizer
importlib.reload(core.sign_tokenizer)

# Import sign tokenizer (NEW - from module)
from core.sign_tokenizer import (
    tokenize_signs,
    generate_sign_ngrams,
    build_cover_from_w,
    query_signs,
)

print("✓ Core modules imported")

✓ Core modules imported


## 2. Compare Hash vs Vocabulary Schemes

In [3]:
# Hash-based scheme (default for English)
hash_scheme = HashIdentifierScheme(p_bits=10, h_bits=32)

# Test with English words
english_words = ["neural", "network", "learning", "deep"]
print("Hash-based indices (English):")
for word in english_words:
    idx = hash_scheme.to_index(word)
    reg, zeros = hash_scheme.to_reg_zeros(word)
    print(f"  '{word}' → reg={reg}, zeros={zeros} → idx={idx}")

print(f"\nIndex range: 0 to {hash_scheme.index_range()}")

Hash-based indices (English):
  'neural' → reg=279, zeros=3 → idx=6420
  'network' → reg=129, zeros=1 → idx=2968
  'learning' → reg=878, zeros=1 → idx=20195
  'deep' → reg=511, zeros=1 → idx=11754

Index range: 0 to 23552


In [4]:
# Vocabulary-based scheme for Chinese
# Build a sample vocabulary
chinese_chars = [
    "你", "好", "我", "是", "人",  # Common characters
    "机", "器", "学", "习",        # Machine learning
    "神", "经", "网", "络",        # Neural network
    "深", "度",                    # Deep
    "数", "据", "处", "理",        # Data processing
]

vocab_scheme = VocabularyIdentifierScheme()
for char in chinese_chars:
    vocab_scheme.add_sign(char)

print("Vocabulary-based indices (Chinese):")
for char in chinese_chars[:10]:
    idx = vocab_scheme.to_index(char)
    print(f"  '{char}' → idx={idx}")

print(f"\nVocabulary size: {vocab_scheme.index_range()}")

Vocabulary-based indices (Chinese):
  '你' → idx=1
  '好' → idx=2
  '我' → idx=3
  '是' → idx=4
  '人' → idx=5
  '机' → idx=6
  '器' → idx=7
  '学' → idx=8
  '习' → idx=9
  '神' → idx=10

Vocabulary size: 20


In [5]:
# Key difference: Hash collisions vs direct lookup
print("Hash scheme - same content ALWAYS same index:")
print(f"  'neural' → {hash_scheme.to_index('neural')}")
print(f"  'neural' → {hash_scheme.to_index('neural')}  (same)")
print(f"  'Neural' → {hash_scheme.to_index('Neural')}  (different! case sensitive)")

print("\nVocabulary scheme - direct lookup:")
print(f"  '神' → {vocab_scheme.to_index('神')}")
print(f"  '神' → {vocab_scheme.to_index('神')}  (same)")
print(f"  '未知' → {vocab_scheme.to_index('未知')}  (unknown = -1)")

Hash scheme - same content ALWAYS same index:
  'neural' → 6420
  'neural' → 6420  (same)
  'Neural' → 7475  (different! case sensitive)

Vocabulary scheme - direct lookup:
  '神' → 10
  '神' → 10  (same)
  '未知' → -1  (unknown = -1)


## 3. Create LookupTable with Vocabulary Scheme

In [6]:
# Create config and LUT with vocabulary scheme
config = Sparse3DConfig(p_bits=10, max_n=3)

# LUT with vocabulary-based identification
lut_vocab = LookupTable(config, identifier_scheme=vocab_scheme)

# Add some Chinese n-grams
chinese_ngrams = [
    ("机",),              # 1-gram: machine
    ("器",),              # 1-gram: device  
    ("学",),              # 1-gram: learn
    ("习",),              # 1-gram: practice
    ("机", "器"),         # 2-gram: machine
    ("学", "习"),         # 2-gram: learning
    ("机", "器", "学"),   # 3-gram: machine learn
]

print("Adding Chinese n-grams to LUT:")
for ngram in chinese_ngrams:
    idx = lut_vocab.add_ntoken(ngram)
    display = ''.join(ngram)
    print(f"  '{display}' (n={len(ngram)}) → idx={idx}")

print(f"\nLUT entries: {len(lut_vocab.ntoken_to_index)}")

Adding Chinese n-grams to LUT:
  '机' (n=1) → idx=6
  '器' (n=1) → idx=7
  '学' (n=1) → idx=8
  '习' (n=1) → idx=9
  '机器' (n=2) → idx=138
  '学习' (n=2) → idx=202
  '机器学' (n=3) → idx=5739

LUT entries: 7


In [7]:
# Compare with hash-based LUT
lut_hash = LookupTable(config)  # Uses default hash scheme

print("Same n-grams with hash-based LUT:")
for ngram in chinese_ngrams:
    idx = lut_hash.add_ntoken(ngram)
    display = ''.join(ngram)
    print(f"  '{display}' (n={len(ngram)}) → idx={idx}")

print("\nNote: Hash indices are much larger (sparse)")
print("Vocabulary indices are compact (dense)")

Same n-grams with hash-based LUT:
  '机' (n=1) → idx=3037
  '器' (n=1) → idx=2004
  '学' (n=1) → idx=17296
  '习' (n=1) → idx=5060
  '机器' (n=2) → idx=10557
  '学习' (n=2) → idx=8695
  '机器学' (n=3) → idx=4077

Note: Hash indices are much larger (sparse)
Vocabulary indices are compact (dense)


## 4. Sign Tokenizer (Using Module)

Uninflected sign systems (Chinese, musical notation, emoji, etc.) don't use spaces - each character/symbol is a semantic unit.
The `tokenize_signs()` and `generate_sign_ngrams()` functions handle this automatically.

In [8]:
# Tokenization uses module functions (not inline definitions)
# tokenize_signs() and generate_sign_ngrams() are imported from core.sign_tokenizer

# Test with Chinese sentence
chinese_text = "机器学习很有趣"
chars = tokenize_signs(chinese_text)
ngrams = generate_sign_ngrams(chars, max_n=3)

print(f"Text: {chinese_text}")
print(f"Characters: {chars}")
print(f"\nN-grams ({len(ngrams)} total):")
for ng in ngrams:
    if ng in (START, END):
        print(f"  {ng}")
    else:
        print(f"  {''.join(ng)} (n={len(ng)})")

Text: 机器学习很有趣
Characters: ['机', '器', '学', '习', '很', '有', '趣']

N-grams (20 total):
  ('<START>',)
  机 (n=1)
  机器 (n=2)
  机器学 (n=3)
  器 (n=1)
  器学 (n=2)
  器学习 (n=3)
  学 (n=1)
  学习 (n=2)
  学习很 (n=3)
  习 (n=1)
  习很 (n=2)
  习很有 (n=3)
  很 (n=1)
  很有 (n=2)
  很有趣 (n=3)
  有 (n=1)
  有趣 (n=2)
  趣 (n=1)
  ('<END>',)


## 5. Build Chinese Vocabulary from Text

In [9]:
# Sample corpus - Chinese is just one example of uninflected sign systems
# (Also works with: musical notation, emoji, mathematical symbols, etc.)
chinese_corpus = [
    "机器学习是人工智能的重要分支",
    "深度学习使用神经网络",
    "数据是机器学习的基础",
    "神经网络可以学习复杂模式",
    "人工智能改变世界",
]

# Build vocabulary from corpus using module functions
all_chars = set()
for text in chinese_corpus:
    chars = tokenize_signs(text)  # Using module function
    all_chars.update(chars)

print(f"Unique signs in corpus: {len(all_chars)}")
print(f"Signs: {''.join(sorted(all_chars))}")

Unique signs in corpus: 36
Signs: 世习人以使分变可器基复学工度式据支改数是智机杂模深用界的础神经络网能要重


In [10]:
# Create vocabulary scheme from corpus
corpus_vocab = VocabularyIdentifierScheme()
for char in sorted(all_chars):
    corpus_vocab.add_sign(char)

print(f"Vocabulary size: {corpus_vocab.index_range()}")
print("\nSample mappings:")
for char in list(all_chars)[:10]:
    print(f"  '{char}' → {corpus_vocab.to_index(char)}")

Vocabulary size: 37

Sample mappings:
  '能' → 34
  '界' → 27
  '重' → 36
  '的' → 28
  '据' → 16
  '复' → 11
  '度' → 14
  '模' → 24
  '是' → 20
  '础' → 29


## 6. ManifoldOS with Vocabulary Scheme

Now let's integrate the vocabulary scheme with ManifoldOS.

In [11]:
# Create config
config = Sparse3DConfig(p_bits=10, max_n=3)

# Build vocabulary from corpus
vocab = VocabularyIdentifierScheme()
for text in chinese_corpus:
    for char in tokenize_signs(text):  # Using module function
        vocab.add_sign(char)

# Create LUT with vocabulary scheme
lut = LookupTable(config, identifier_scheme=vocab)

print(f"Vocabulary: {vocab.index_range()} characters")
print(f"LUT ready with vocabulary scheme")

Vocabulary: 37 characters
LUT ready with vocabulary scheme


In [12]:
# Process sign text manually to see the indices
test_text = "深度学习"
chars = tokenize_signs(test_text)  # Using module function
ngrams = generate_sign_ngrams(chars, max_n=3)  # Using module function

print(f"Text: {test_text}")
print(f"\nN-grams with vocabulary indices:")
for ng in ngrams:
    idx = lut.add_ntoken(ng)
    if ng in (START, END):
        display = str(ng)
    else:
        display = ''.join(ng)
    print(f"  '{display}' → idx={idx}")

print(f"\nLUT now has {len(lut.ntoken_to_index)} entries")

Text: 深度学习

N-grams with vocabulary indices:
  '('<START>',)' → idx=-1
  '深' → idx=15
  '深度' → idx=586
  '深度学' → idx=132984
  '度' → idx=16
  '度学' → idx=267
  '度学习' → idx=19618
  '学' → idx=3
  '学习' → idx=106
  '习' → idx=4
  '('<END>',)' → idx=-1

LUT now has 11 entries


## 7. Build AM/W with Vocabulary Indices

In [13]:
# Create edges from n-gram sequence
def create_edges_from_ngrams(ngrams: list, lut: LookupTable) -> list:
    """Create edges from sequential n-grams."""
    edges = []
    for i in range(len(ngrams) - 1):
        row_ng = ngrams[i]
        col_ng = ngrams[i + 1]
        
        row_idx = lut.get_ntoken_index(row_ng)
        col_idx = lut.get_ntoken_index(col_ng)
        
        if row_idx is not None and col_idx is not None:
            # Layer is determined by the target n-gram
            layer = 0 if col_ng in (START, END) else len(col_ng) - 1
            layer = min(layer, config.max_n - 1)
            edges.append(Edge3D(n=layer, row=row_idx, col=col_idx, value=1.0))
    
    return edges

# Process all corpus using module functions
all_edges = []
for text in chinese_corpus:
    chars = tokenize_signs(text)  # Module function
    ngrams = generate_sign_ngrams(chars, max_n=3)  # Module function
    
    # Add all n-grams to LUT first
    for ng in ngrams:
        lut.add_ntoken(ng)
    
    # Create edges
    edges = create_edges_from_ngrams(ngrams, lut)
    all_edges.extend(edges)

print(f"Total edges: {len(all_edges)}")
print(f"LUT entries: {len(lut.ntoken_to_index)}")

Total edges: 152
LUT entries: 114


In [14]:
# Build AM from edges using from_edges (efficient batch construction)
am = SparseAM3D.from_edges(config, all_edges)

print(f"AM built with vocabulary indices:")
print(f"  Total edges (nnz): {am.nnz}")
print(f"  Layers: {am.config.max_n}")

# Check layer statistics
for n in range(config.max_n):
    rows, cols = am.layer_active(n)
    print(f"  L{n}: {len(rows)} rows, {len(cols)} cols")

/home/alexmy/SGS/SGS_lib/fractal_manifold/fractal_manifold/.venv/lib/python3.13/site-packages/torch/cuda/__init__.py:435: UserWarning: 
    Found GPU1 Quadro M1200 which is of cuda capability 5.0.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (7.0) - (12.0)
    
  queued_call()
/home/alexmy/SGS/SGS_lib/fractal_manifold/fractal_manifold/.venv/lib/python3.13/site-packages/torch/cuda/__init__.py:435: UserWarning: 
    Please install PyTorch with a following CUDA
    configurations:  12.6 following instructions at
    https://pytorch.org/get-started/locally/
    
  queued_call()
/home/alexmy/SGS/SGS_lib/fractal_manifold/fractal_manifold/.venv/lib/python3.13/site-packages/torch/cuda/__init__.py:435: UserWarning: 
Quadro M1200 with CUDA capability sm_50 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_70 sm_75 sm_80 sm_86 sm_90 sm_100 sm_120.
If you want to use the Quadro M1200 GPU with Py

AM built with vocabulary indices:
  Total edges (nnz): 152
  Layers: 3
  L0: 49 rows, 37 cols
  L1: 32 rows, 38 cols
  L2: 34 rows, 38 cols


In [15]:
# Build W transition matrix
from core.mf_algebra import build_w_from_am

W = build_w_from_am(am, config)

# Count transitions
total_transitions = sum(
    sum(len(targets) for targets in layer.values())
    for layer in W.values()
)

print(f"W matrix built:")
print(f"  Total transitions: {total_transitions}")
print(f"  Source indices: {len(W)}")

W matrix built:
  Total transitions: 129
  Source indices: 3


In [16]:
# Trace a path through W
print("Sample transitions from W:")

# Find a starting point
start_idx = lut.get_ntoken_index(START)
if start_idx and start_idx in W:
    print(f"\nFrom START ({start_idx}):")
    for layer, targets in W[start_idx].items():
        for target_idx, prob in list(targets.items())[:5]:
            # Recover n-token from index
            ntokens = lut.get_ntokens_at_index(target_idx)
            for _, nt in ntokens:
                if nt not in (START, END):
                    display = ''.join(nt)
                    print(f"  → '{display}' (L{layer}, prob={prob:.3f})")

Sample transitions from W:


## 8. Query with Vocabulary Scheme

In [17]:
# =============================================================================
# PROPER QUERY WORKFLOW:
# 1. Query MUST be ingested first (or use existing corpus indices)
# 2. Build query HLLSet from query indices
# 3. Build cover HLLSet from W transitions 
# 4. Disambiguate cover to get relevant indices
# 5. LUT lookup to recover tokens
# 6. Order by AM layer structure
# =============================================================================

from core.mf_algebra import cascading_disambiguate, LayerHLLSets

# First, let's verify what's in our structures
print("=== Corpus Ingested ===")
print(f"AM edges: {am.nnz}")
print(f"LUT entries: {len(lut.index_to_ntokens)}")
print(f"W source nodes: {sum(len(layer) for layer in W.values())}")

# Build LayerHLLSets from our AM (needed for disambiguation)
layer_hllsets = LayerHLLSets.from_am(am, p_bits=config.p_bits)

# Get START index for proper traversal
start_idx = lut.get_ntoken_index(START)
print(f"\nSTART index in LUT: {start_idx}")

# Show layer distribution
for n in range(config.max_n):
    rows, cols = am.layer_active(n)
    print(f"Layer {n}: {len(rows)} unique rows, {len(cols)} unique cols")

=== Corpus Ingested ===
AM edges: 152
LUT entries: 113
W source nodes: 115

START index in LUT: -1
Layer 0: 49 unique rows, 37 unique cols
Layer 1: 32 unique rows, 38 unique cols
Layer 2: 34 unique rows, 38 unique cols


In [18]:
# =============================================================================
# Build cover from W using INTERSECTION of row and column covers
# Using module functions: build_cover_from_w()
# =============================================================================

# Test with different strategies (functions imported from core.sign_tokenizer)
print("=== Cover Building Strategies ===")
test_query = {1, 2, 3, 4}  # 机, 器, 学, 习

for strategy in ['rows_only', 'cols_only', 'union', 'intersection']:
    cover = build_cover_from_w(test_query, W, max_depth=2, strategy=strategy)
    print(f"  {strategy}: {len(cover)} indices")

print(f"\nAll indices in AM: {len(am.all_active_rows | am.all_active_cols)}")
print("\nKey insight: INTERSECTION gives smaller, more precise cover!")
print("(Indices that are BOTH reachable from AND can reach the query)")

=== Cover Building Strategies ===
  rows_only: 21 indices
  cols_only: 25 indices
  union: 33 indices
  intersection: 13 indices

All indices in AM: 113

Key insight: INTERSECTION gives smaller, more precise cover!
(Indices that are BOTH reachable from AND can reach the query)


In [19]:
# =============================================================================
# COMPLETE QUERY PIPELINE (Using module function: query_signs())
# =============================================================================

# The query_signs() function from core.sign_tokenizer implements:
# 1. Tokenize query
# 2. Generate ALL n-grams (1-gram, 2-gram, 3-gram) from query
# 3. Look up indices for ALL n-grams
# 4. Build cover from W using row/col INTERSECTION
# 5. Disambiguate cover
# 6. Recover tokens and order by layer

print("Testing query_signs() from module:")
results = query_signs(
    query_text="机器学习",
    lut=lut,
    am=am,
    W=W,
    layer_hllsets=layer_hllsets,
    top_k=10,
    cover_strategy='intersection',
    verbose=True
)

print(f"\nReturned {len(results)} results")

Testing query_signs() from module:
Query: 机器学习 (strategy=intersection)
  Signs: 4, Indices: 9 (from 9 n-grams)
  Cover: rows=21, cols=25, intersection=13, using=13
  Results: 10 tokens
    L2: ['器学习', '学习是', '学习的']...
    L1: ['机器']
    L0: ['机', '器', '学']...

Returned 10 results


In [20]:
# =============================================================================
# COMPARISON: Intersection vs Union Cover Strategies
# =============================================================================

print("=" * 70)
print("COMPARISON: Intersection vs Union Cover Strategies")
print("=" * 70)

test_query = "机器学习"

print(f"\nQuery: {test_query} with INTERSECTION (precise)")
result_int = query_signs(test_query, lut, am, W, layer_hllsets, 
                         top_k=10, cover_strategy='intersection', verbose=True)

print(f"\n{'='*70}")
print(f"Query: {test_query} with UNION (broad)")
result_uni = query_signs(test_query, lut, am, W, layer_hllsets, 
                         top_k=10, cover_strategy='union', verbose=True)

# Summary
print(f"\n{'='*70}")
print("SUMMARY:")
print(f"  Intersection: {len(result_int)} results (more precise)")
print(f"  Union: {len(result_uni)} results (broader recall)")

COMPARISON: Intersection vs Union Cover Strategies

Query: 机器学习 with INTERSECTION (precise)
Query: 机器学习 (strategy=intersection)
  Signs: 4, Indices: 9 (from 9 n-grams)
  Cover: rows=21, cols=25, intersection=13, using=13
  Results: 10 tokens
    L2: ['器学习', '学习是', '学习的']...
    L1: ['机器']
    L0: ['机', '器', '学']...

Query: 机器学习 with UNION (broad)
Query: 机器学习 (strategy=union)
  Signs: 4, Indices: 9 (from 9 n-grams)
  Cover: rows=21, cols=25, intersection=13, using=33
  Results: 22 tokens
    L2: ['是机器', '学习是', '学习的']...
    L1: ['机器', '器学', '度学']...
    L0: ['机', '器', '学']...

SUMMARY:
  Intersection: 10 results (more precise)
  Union: 10 results (broader recall)


In [21]:
# =============================================================================
# REBUILD CLEAN: LUT, AM, W with ordered vocabulary
# =============================================================================

# Build vocabulary in corpus order (deterministic)
chars_in_order = []
for text in chinese_corpus:
    for char in tokenize_signs(text):
        if char not in chars_in_order:
            chars_in_order.append(char)

vocab_dict = {char: idx + 1 for idx, char in enumerate(chars_in_order)}
corpus_vocab = VocabularyIdentifierScheme(vocabulary=vocab_dict)

print(f"Vocabulary size: {len(vocab_dict)} signs")
print(f"Sample: {list(vocab_dict.items())[:5]}")

# Rebuild LUT
new_config = Sparse3DConfig(p_bits=10, max_n=3, h_bits=32)
lut = LookupTable(config=new_config, identifier_scheme=corpus_vocab)

# Add all n-grams
for text in chinese_corpus:
    chars = tokenize_signs(text)
    ngrams = generate_sign_ngrams(chars)
    for ng in ngrams:
        lut.add_ntoken(ng)

print(f"LUT: {len(lut.index_to_ntokens)} unique indices")

Vocabulary size: 36 signs
Sample: [('机', 1), ('器', 2), ('学', 3), ('习', 4), ('是', 5)]
LUT: 113 unique indices


In [22]:
# Build edges between consecutive n-grams
all_edges = []

for text in chinese_corpus:
    chars = tokenize_signs(text)
    ngrams = generate_sign_ngrams(chars)
    
    prev_by_layer = {0: None, 1: None, 2: None}
    for ng in ngrams:
        if ng in (START, END):
            continue
        layer = len(ng) - 1
        idx = lut.get_ntoken_index(ng)
        
        if prev_by_layer[layer] is not None:
            prev_idx = prev_by_layer[layer]
            all_edges.append(Edge3D(n=layer, row=prev_idx, col=idx, value=1.0))
        prev_by_layer[layer] = idx

# Add START edges
for text in chinese_corpus:
    chars = tokenize_signs(text)
    if chars:
        first_idx = lut.get_ntoken_index((chars[0],))
        start_idx = lut.get_ntoken_index(START)
        all_edges.append(Edge3D(n=0, row=start_idx, col=first_idx, value=1.0))

# Build AM
am = SparseAM3D.from_edges(new_config, all_edges)
print(f"AM: {am.nnz} edges")
for n in range(new_config.max_n):
    rows, cols = am.layer_active(n)
    print(f"  L{n}: {len(rows)} rows, {len(cols)} cols")

AM: 137 edges
  L0: 33 rows, 36 cols
  L1: 34 rows, 36 cols
  L2: 34 rows, 36 cols


In [23]:
# Build W and LayerHLLSets
W = build_w_from_am(am, new_config)
layer_hllsets = LayerHLLSets.from_am(am, p_bits=new_config.p_bits)

print(f"W: {sum(len(l) for l in W.values())} sources")
print(f"LayerHLLSets ready for disambiguation")

W: 101 sources
LayerHLLSets ready for disambiguation


In [24]:
# =============================================================================
# FINAL TEST: Multiple queries
# =============================================================================

print("=" * 70)
print("FINAL TEST: Query Pipeline with Different Sign Sequences")
print("=" * 70)

# Test 1: 学习 (learning)
result1 = query_signs("学习", lut, am, W, layer_hllsets, top_k=10, verbose=True)

print("\n" + "=" * 70)

# Test 2: 机器学习 (machine learning)
result2 = query_signs("机器学习", lut, am, W, layer_hllsets, top_k=10, verbose=True)

print("\n" + "=" * 70)

# Test 3: 神经网络 (neural network)
result3 = query_signs("神经网络", lut, am, W, layer_hllsets, top_k=10, verbose=True)

FINAL TEST: Query Pipeline with Different Sign Sequences
Query: 学习 (strategy=intersection)
  Signs: 2, Indices: 3 (from 3 n-grams)
  Cover: rows=21, cols=15, intersection=4, using=4
  Results: 4 tokens
    L1: ['学习']
    L0: ['机', '学', '习']

Query: 机器学习 (strategy=intersection)
  Signs: 4, Indices: 9 (from 9 n-grams)
  Cover: rows=30, cols=24, intersection=10, using=10
  Results: 8 tokens
    L2: ['器学习']
    L1: ['机器', '学习']
    L0: ['机', '器', '学']...

Query: 神经网络 (strategy=intersection)
  Signs: 4, Indices: 9 (from 9 n-grams)
  Cover: rows=15, cols=16, intersection=9, using=9
  Results: 7 tokens
    L2: ['经网络']
    L1: ['神经', '网络']
    L0: ['神', '经', '网']...


## 9. Summary: Hash vs Vocabulary Schemes

| Aspect | HashIdentifierScheme | VocabularyIdentifierScheme |
|--------|---------------------|---------------------------|
| **Use case** | Inflected languages (English, Russian, German) | Uninflected sign systems |
| **Index range** | Large (sparse) | Compact (dense) |
| **Collisions** | Possible (rare) | None (direct lookup) |
| **Unknown tokens** | Hash to some index | Return -1 (explicit unknown) |
| **Examples** | English words, sentences | Chinese, Musical notes (♩♪♫♬), Emoji (🎵🎶), Math symbols |

### Key Insight

The **AM and W structures don't care** how indices are computed. They just need:
1. Consistent mapping: same content → same index
2. Deterministic: reproducible across sessions

This enables the fractal manifold to work with **ANY sign system**!

### Module Functions Used

From `core.sign_tokenizer`:
- `tokenize_signs()` - Tokenize uninflected sign systems (any Unicode)
- `generate_sign_ngrams()` - Generate 1-grams, 2-grams, 3-grams
- `build_cover_from_w()` - Build cover using row/col intersection
- `query_signs()` - Complete query pipeline

In [25]:
# Final comparison: Hash vs Vocabulary schemes
print("=" * 50)
print("SCHEME COMPARISON: Hash vs Vocabulary")
print("=" * 50)

test_content = "机器学习"
chars = tokenize_signs(test_content)

print(f"\nContent: {test_content}")
print(f"Signs: {chars}")

print(f"\nHash scheme indices (sparse):")
for char in chars:
    idx = hash_scheme.to_index(char)
    print(f"  '{char}' → {idx}")

print(f"\nVocabulary scheme indices (dense):")
for char in chars:
    idx = corpus_vocab.to_index(char)
    print(f"  '{char}' → {idx}")

print(f"\nHash range: 0-{hash_scheme.index_range()} (2^32 * 1024)")
print(f"Vocab range: 0-{corpus_vocab.index_range()} (vocabulary size)")
print("\n✓ Same HRT structure, different identification schemes!")

SCHEME COMPARISON: Hash vs Vocabulary

Content: 机器学习
Signs: ['机', '器', '学', '习']

Hash scheme indices (sparse):
  '机' → 3037
  '器' → 2004
  '学' → 17296
  '习' → 5060

Vocabulary scheme indices (dense):
  '机' → 1
  '器' → 2
  '学' → 3
  '习' → 4

Hash range: 0-23552 (2^32 * 1024)
Vocab range: 0-37 (vocabulary size)

✓ Same HRT structure, different identification schemes!


## 11. HashVocabularyScheme: Precise Hash-Based Identification

**NEW**: `HashVocabularyScheme` combines benefits of both schemes:
- Uses **full hash** as index (not (reg, zeros) compressed)
- Each sign gets **exact** row/column in AM/W
- Context unions are precise (neighbors of 你 = exactly tokens seen with 你)
- Still stores (reg, zeros) for HLLSet compatibility

| Scheme | Pipeline | Index Range | AM/W Precision |
|--------|----------|-------------|----------------|
| `HashIdentifierScheme` | token → hash → (reg,zeros) → index | ~32K | Cloud (compression) |
| `HashVocabularyScheme` | sign → hash → index | ~4B | **Exact** (per-sign) |
| `VocabularyIdentifierScheme` | sign → sequential → index | vocab size | Exact but arbitrary |

In [26]:
# Import HashVocabularyScheme
from core.mf_algebra import HashVocabularyScheme

# Build vocabulary with precise hash-based indices
hvs = HashVocabularyScheme(p_bits=10, h_bits=32)

# Add Chinese characters (lazy buildup)
for char in chinese_chars:
    hvs.add_sign(char)

print("HashVocabularyScheme indices (precise):")
print("=" * 60)
for char in chinese_chars[:8]:
    hvs_idx = hvs.to_index(char)
    his_idx = hvs.to_his_index(char)  # HIS-compatible (compressed)
    reg, zeros = hvs.to_reg_zeros(char)
    print(f"  '{char}' → HVS: {hvs_idx:>10} | HIS: {his_idx:>5} | (reg={reg}, zeros={zeros})")

print(f"\nHVS index range: {hvs.index_range():,} (full 32-bit hash)")
print(f"HIS index range: {hvs.his_index_range():,} (compressed)")

# Collision report
report = hvs.collision_report()
print(f"\nCollision report:")
print(f"  Vocabulary size: {report['vocabulary_size']}")
print(f"  Unique indices: {report['unique_indices']}")
print(f"  Collisions: {report['collisions']}")

HashVocabularyScheme indices (precise):
  '你' → HVS: 1446033542 | HIS:  3083 | (reg=134, zeros=1)
  '好' → HVS:  669318732 | HIS: 13524 | (reg=588, zeros=0)
  '我' → HVS:  888275388 | HIS: 10219 | (reg=444, zeros=7)
  '是' → HVS:  806750753 | HIS: 12536 | (reg=545, zeros=1)
  '人' → HVS: 1225946906 | HIS: 18262 | (reg=794, zeros=0)
  '机' → HVS:  225695876 | HIS:  3037 | (reg=132, zeros=1)
  '器' → HVS: 2837930071 | HIS:  2004 | (reg=87, zeros=3)
  '学' → HVS: 2088371952 | HIS: 17296 | (reg=752, zeros=0)

HVS index range: 4,294,967,296 (full 32-bit hash)
HIS index range: 23,552 (compressed)

Collision report:
  Vocabulary size: 19
  Unique indices: 19
  Collisions: 0


## 12. HVS ↔ HIS Interoperability

With same hash function, p_bits, and seed, HVS and HIS are fully compatible:

```
HVS: sign → hash → index (precise, ~4B range)
       ↓
      P(hash), Zeros(hash)
       ↓
HIS: sign → hash → (reg, zeros) → index (compressed, ~32K range)
```

This enables **mixing precise and compressed data**.

In [27]:
# HVS → HIS projection maps
print("HVS → HIS Projection Map")
print("=" * 60)

# Build projection map
hvs_to_his = hvs.hvs_to_his_index_map()
his_to_hvs = hvs.his_to_hvs_index_map()

print(f"HVS indices mapped to HIS: {len(hvs_to_his)} entries")
print(f"Unique HIS indices: {len(his_to_hvs)} entries")

# Show some mappings
print("\nSample mappings (HVS → HIS):")
for char in chinese_chars[:5]:
    hvs_idx = hvs.to_index(char)
    his_idx = hvs_to_his[hvs_idx]
    print(f"  '{char}': {hvs_idx} → {his_idx}")

# Check for HIS index collisions (multiple HVS indices → same HIS index)
collisions = {k: v for k, v in his_to_hvs.items() if len(v) > 1}
if collisions:
    print(f"\n⚠️ HIS collisions found: {len(collisions)}")
    for his_idx, hvs_set in list(collisions.items())[:3]:
        signs = [hvs.get_sign(idx) for idx in hvs_set]
        print(f"  HIS {his_idx} ← signs: {signs}")
else:
    print("\n✓ No HIS collisions (vocabulary is small enough)")

HVS → HIS Projection Map
HVS indices mapped to HIS: 19 entries
Unique HIS indices: 19 entries

Sample mappings (HVS → HIS):
  '你': 1446033542 → 3083
  '好': 669318732 → 13524
  '我': 888275388 → 10219
  '是': 806750753 → 12536
  '人': 1225946906 → 18262

✓ No HIS collisions (vocabulary is small enough)


## 13. Vocabulary as HLLSet Fingerprint

Vocabulary can be represented as an HLLSet for:
- O(1) vocabulary comparison via tau similarity
- Cross-installation vocabulary matching
- Social/domain profiling by vocabulary overlap

In [28]:
# Get vocabulary as HLLSet fingerprint
vocab_hllset = hvs.to_hllset()
print("Vocabulary HLLSet Fingerprint")
print("=" * 60)
print(f"Fingerprint shape: {vocab_hllset.shape}")
print(f"Non-zero registers: {(vocab_hllset > 0).sum()}")
print(f"Estimated cardinality: {hvs.vocabulary_cardinality():.0f}")
print(f"Actual vocabulary size: {len(hvs.known_signs)}")

# Create another vocabulary for comparison
hvs_tech = HashVocabularyScheme(p_bits=10, h_bits=32)
for char in ["机", "器", "学", "习", "数", "据", "算", "法"]:  # Tech vocab
    hvs_tech.add_sign(char)

hvs_social = HashVocabularyScheme(p_bits=10, h_bits=32)
for char in ["你", "好", "我", "是", "人", "说", "话", "听"]:  # Social vocab
    hvs_social.add_sign(char)

# Compare vocabularies
print(f"\nVocabulary comparison:")
print(f"  Tech vocab size: {len(hvs_tech.known_signs)}")
print(f"  Social vocab size: {len(hvs_social.known_signs)}")

# Tau similarity (O(1) via HLLSet)
tau_hvs_tech = hvs.tau(hvs_tech)
tau_hvs_social = hvs.tau(hvs_social)
tau_tech_social = hvs_tech.tau(hvs_social)

print(f"\nTau similarity (O(1) via HLLSet):")
print(f"  Main ↔ Tech: {tau_hvs_tech:.3f}")
print(f"  Main ↔ Social: {tau_hvs_social:.3f}")
print(f"  Tech ↔ Social: {tau_tech_social:.3f}")

Vocabulary HLLSet Fingerprint
Fingerprint shape: (1024,)
Non-zero registers: 19
Estimated cardinality: 19
Actual vocabulary size: 19

Vocabulary comparison:
  Tech vocab size: 8
  Social vocab size: 8

Tau similarity (O(1) via HLLSet):
  Main ↔ Tech: 0.750
  Main ↔ Social: 0.625
  Tech ↔ Social: 0.000


In [29]:
# Vocabulary operations
print("Vocabulary Set Operations")
print("=" * 60)

# Intersection - signs in both
shared_tech = hvs.vocabulary_intersection(hvs_tech)
shared_social = hvs.vocabulary_intersection(hvs_social)

print(f"Main ∩ Tech: {shared_tech}")
print(f"Main ∩ Social: {shared_social}")

# Difference - signs in main but not in other
unique_from_tech = hvs.vocabulary_diff(hvs_tech)
unique_from_social = hvs.vocabulary_diff(hvs_social)

print(f"\nMain \\ Tech: {len(unique_from_tech)} signs")
print(f"Main \\ Social: {len(unique_from_social)} signs")

# Merge vocabularies
merged = hvs_tech.merge_vocabulary(hvs_social)
print(f"\nMerged (Tech ∪ Social): {len(merged.known_signs)} signs")
print(f"  Signs: {list(merged.known_signs.keys())}")

# Jaccard similarity
jaccard_tech_social = hvs_tech.jaccard(hvs_social)
print(f"\nJaccard(Tech, Social): {jaccard_tech_social:.3f} (|A∩B| / |A∪B|)")

Vocabulary Set Operations
Main ∩ Tech: {'数', '器', '据', '机', '习', '学'}
Main ∩ Social: {'我', '你', '好', '人', '是'}

Main \ Tech: 13 signs
Main \ Social: 14 signs

Merged (Tech ∪ Social): 16 signs
  Signs: ['机', '器', '学', '习', '数', '据', '算', '法', '你', '好', '我', '是', '人', '说', '话', '听']

Jaccard(Tech, Social): 0.000 (|A∩B| / |A∪B|)


## 14. Summary: Three Identifier Schemes

| Scheme | Pipeline | Index Range | Use Case |
|--------|----------|-------------|----------|
| `HashIdentifierScheme` | token → hash → (reg,zeros) → index | ~32K | Default, open vocabularies, HLLSet-native |
| `HashVocabularyScheme` | sign → hash → index | ~4B | Compact vocabularies, exact AM/W addressing |
| `VocabularyIdentifierScheme` | sign → sequential → index | vocab size | Custom index assignments |

**Key features of HashVocabularyScheme:**
- ✓ Lazy vocabulary buildup
- ✓ Vocabulary as HLLSet fingerprint (O(1) comparison)
- ✓ $\text{HVS} ↔ \text{HIS}$ interoperability (same hash, compatible addressing)
- ✓ Cross-installation vocabulary matching
- ✓ Social/domain profiling via tau similarity